In [21]:
plot_station = [
    '58566','58567','58564','58569',
    '58556','58555',
    '58560','58654','58657','58658','58763','58656','58761',
    '58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668',#台州
    'K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505'
]
len(plot_station)

42

# sql 

In [ ]:
SELECT * FROM from TAB_StationInfo where (IIiii  IN  ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668','K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505'))

select a.IIiii,b.lat,b.lon,SUM(a.RR)/10.0
from Tab_Aws AS a left join TAB_StationInfo AS b on a.IIiii=b.IIiii 
WHERE (b.IIiii  IN  ('58566','58567','58564','58569','58556','58555','58560','58654') and (tTime BETWEEN '2023-03-10 20:00' and '2023-03-15 08:00')) 
GROUP BY a.IIiii,lat,lon






# 自动站数据查询

In [24]:
%%time
import pymssql 
import pandas as pd

## 连接数据库
server = '172.21.158.201'    # 连接服务器地址
user = 'down'# 连接帐号
password = 'downx'# 连接密码
conn = pymssql.connect(server, user, password, 'ZJSZDZDB')  #获取连接

sleclt_str = '''select a.IIiii,b.lat,b.lon,SUM(a.RR)/10.0 '''
from_str = '''from Tab_Aws AS a left join TAB_StationInfo AS b on a.IIiii=b.IIiii '''
station_Astr = '''('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668')'''
station_Mstr = '''('K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505')'''
where_str1 = '''WHERE (b.IIiii  IN  '''
where_str2 = ''' and '''
time_str = '''(tTime BETWEEN '2023-03-10 20:00' and '2023-03-15 08:00')'''
where_str3 = ''') ''' 
group_str = '''GROUP BY a.IIiii,lat,lon'''

sql_all = sleclt_str + from_str + where_str1 + station_Astr + where_str2 + time_str + where_str3 + group_str 

df = pd.read_sql(sql_all , con=conn)

CPU times: user 7.78 ms, sys: 0 ns, total: 7.78 ms
Wall time: 16.8 ms


# EC数据的查询



In [25]:
%%time
import pymssql 
import pandas as pd

## 连接数据库
server = '172.21.158.201'    # 连接服务器地址
user = 'down'# 连接帐号
password = 'downx'# 连接密码
conn = pymssql.connect(server, user, password, 'ZJSZDZDB')  #获取连接

sql_all = '''
select IIiii,lat,lon FROM TAB_StationInfo
WHERE (IIiii  IN  ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668','K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505'))
'''
df = pd.read_sql(sql_all , con=conn)

CPU times: user 6.92 ms, sys: 515 µs, total: 7.44 ms
Wall time: 22.9 ms


# 编写自动站数据的类

In [75]:
%%time
import pymysql
import pandas as pd
import os
import xarray as xr
import numpy as np

class sql_plot:
    def __init__(self, start_time,end_time,select_type):
        self.start =  start_time 
        self.end = end_time
        self.type = select_type
        self.station = ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668','K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505')
        self.Aws = ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668')
        self.Mws = ('K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505')  
        self.server = '127.0.0.1' 
        self.user = 'root'
        self.password = '051219'# 连接密码   
        self.port = 3306
        self.database = "ZJSZDZDB"
    def return_zdz_data(self):
        self.start ="'" +  self.start + "'" 
        self.end = "'" +  self.end + "'" 
        conn = pymysql.connect(
            host = self.server, 
            port = self.port,
            user = self.user, 
            password = self.password,
            database = self.database)
        sql_Aws = f'''select a.IIiii,b.lat,b.lon,SUM(a.RR)/10.0
            from TAB_Aws2019 AS a left join TAB_StationInfo AS b on a.IIiii=b.IIiii 
            WHERE (b.IIiii  IN  {self.Aws} and (tTime BETWEEN {self.start} and {self.end} )) 
            GROUP BY a.IIiii,lat,lon'''
        sql_Mws = f'''select a.IIiii,b.lat,b.lon,SUM(a.RR)/10.0
            from TAB_Mws2019 AS a left join TAB_StationInfo AS b on a.IIiii=b.IIiii 
            WHERE (b.IIiii  IN  {self.Mws} and (tTime BETWEEN {self.start} and {self.end} )) 
            GROUP BY a.IIiii,lat,lon '''
        dfa = pd.read_sql(sql_Aws, con=conn)
        dfm = pd.read_sql(sql_Mws, con=conn)
        df = pd.concat([dfa,dfm])
        # 输出数据
        data_canvas = {
            "station_list": [],
            "station": []
        }
        for i in range(df.shape[0]):
            station_data = []
            station_data.append(round(df.iloc[i, 2],2))#lon
            station_data.append(round(df.iloc[i, 1],2))#lat
            value = round(df.iloc[i, 3],2)
            if value!=-9999:
                station_data.append(value)
                data_canvas['station_list'].append(df.iloc[i,0])
                data_canvas['station'].append(station_data)
        canvas_data = data_canvas
        return canvas_data
    def return_ec_data(self):
        # 数据---文档
        test_time = '2022041700'
        file_path = "/home/workspace/Data/My_Git/" + test_time + "/"
        # 文件--数据
        conn = pymysql.connect(
            host = self.server, 
            port = self.port,
            user = self.user, 
            password = self.password,
            database = self.database)
        sql_location = f'''
            select IIiii,lat,lon from TAB_StationInfo
            where (IIiii in  {self.station} )     
        '''
        df = pd.read_sql(sql_location, con=conn)
        # 查询ec数据
        files = os.listdir(file_path)  
        fstart = xr.open_dataset(file_path + files[int(self.start)])
        tp_start = fstart.tp
        fend = xr.open_dataset(file_path + files[int(self.end)])
        tp_end = fend.tp 
        tp = tp_end.data - tp_start.data
        # 输出数据
        data_canvas = {
            "station_list": [],
            "station": []
        }
        for i in range(df.shape[0]):
            station_data = []
            
            station_data.append(round(df.iloc[i, 2],2))#lon
            station_data.append(round(df.iloc[i, 1],2))#lat
            # 求数据之差
            tp0 = tp_end.sel(lonS=round(df.iloc[i, 2],2), latS=round(df.iloc[i, 1],2),method='nearest').data.tolist()[0]
            print(type(tp0))
            tp1 = tp_start.sel(lonS=round(df.iloc[i, 2],2), latS=round(df.iloc[i, 1],2),method='nearest').data[0]
            value = round((tp0-tp1),2)
            if not np.isnan(value):
                station_data.append(value)
                data_canvas['station_list'].append(df.iloc[i,0])
                data_canvas['station'].append(station_data)
        canvas_data = data_canvas
        return canvas_data
    def return_data(self):
        if self.type == "zdz":
            canvas_data = self.return_zdz_data()
        else:
            canvas_data = self.return_ec_data()
        return canvas_data

# zdz    
#start_time,end_time,select_type = "2019-03-10 20:00","2019-03-15 20:00","zdz"
# ec
start_time,end_time,select_type = 1,25,"ec"
sql_worker = sql_plot(start_time,end_time,select_type)
data = sql_worker.return_data()

CPU times: user 139 ms, sys: 344 µs, total: 140 ms
Wall time: 161 ms


In [73]:
data

{'station_list': [58559,
  58568,
  58652,
  58653,
  58660,
  58661,
  58662,
  58663,
  58664,
  58665,
  58666,
  58667,
  58668,
  58669,
  'K8201',
  'K8217',
  'K8282',
  'K8301',
  'K8413',
  'K8505',
  'K8609',
  'K8611',
  'K8705',
  'K8706',
  'K8818',
  'K8821',
  'K8903',
  'K8910'],
 'station': [[120.97, 29.15, 21.4],
  [121.38, 29.12, 22.3],
  [120.72, 28.87, 19.1],
  [120.92, 28.81, 22.5],
  [121.19, 28.87, 24.0],
  [121.51, 28.66, 23.9],
  [121.71, 29.01, 28.7],
  [121.92, 28.72, 20.3],
  [121.36, 28.37, 31.6],
  [121.42, 28.62, 25.5],
  [121.9, 28.45, 10.0],
  [121.27, 28.08, 18.7],
  [121.52, 28.1, 21.1],
  [121.62, 28.26, 23.9],
  [121.28, 28.65, 26.8],
  [121.17, 28.63, 24.9],
  [120.98, 28.59, 24.3],
  [121.35, 28.58, 29.3],
  [121.56, 28.45, 27.8],
  [121.24, 28.13, 29.6],
  [121.45, 28.85, 28.9],
  [121.64, 28.71, 28.3],
  [120.79, 29.05, 0.6],
  [121.0, 29.07, 23.7],
  [121.31, 29.0, 27.9],
  [121.58, 28.93, 30.8],
  [120.79, 28.72, 20.0],
  [120.34, 28.7, 21.7]

In [36]:
import pymysql
import pandas as pd
import os
import xarray as xr
test_time = '2022041700'
file_path = "/home/workspace/Data/My_Git/" + test_time + "/"
files = os.listdir(file_path)  
print(files)
fstart = xr.open_dataset(file_path + files[15])

['ecfine.I2022041700.000.F2022041700.nc', 'ecfine.I2022041700.003.F2022041703.nc', 'ecfine.I2022041700.006.F2022041706.nc', 'ecfine.I2022041700.009.F2022041709.nc', 'ecfine.I2022041700.012.F2022041712.nc', 'ecfine.I2022041700.015.F2022041715.nc', 'ecfine.I2022041700.018.F2022041718.nc', 'ecfine.I2022041700.021.F2022041721.nc', 'ecfine.I2022041700.024.F2022041800.nc', 'ecfine.I2022041700.027.F2022041803.nc', 'ecfine.I2022041700.030.F2022041806.nc', 'ecfine.I2022041700.033.F2022041809.nc', 'ecfine.I2022041700.036.F2022041812.nc', 'ecfine.I2022041700.039.F2022041815.nc', 'ecfine.I2022041700.042.F2022041818.nc', 'ecfine.I2022041700.045.F2022041821.nc', 'ecfine.I2022041700.048.F2022041900.nc', 'ecfine.I2022041700.051.F2022041903.nc', 'ecfine.I2022041700.054.F2022041906.nc', 'ecfine.I2022041700.057.F2022041909.nc', 'ecfine.I2022041700.060.F2022041912.nc', 'ecfine.I2022041700.063.F2022041915.nc', 'ecfine.I2022041700.066.F2022041918.nc', 'ecfine.I2022041700.069.F2022041921.nc', 'ecfine.I202204

In [17]:
import pymssql 
import pandas as pd

## 连接数据库
server = '172.21.158.201'    # 连接服务器地址
user = 'down'# 连接帐号
password = 'downx'# 连接密码
conn = pymssql.connect(server, user, password, 'ZJSZDZDB')  #获取连接

## 以查询区域站数据为例 ---- 以下是说明（国家站对应TAB_Mws2022改为TAB_Aws2022）
# RR是小时降水，R01是第一分钟的降水,以此类推
# TAB_Mws2022是对应每年的数据表，具体2022可以更改对应年份------》“from TAB_Mws2022 as a left join TAB_StationInfo”
# TAB_Mws2022是2022年区域站的数据，若想查询国家站数据则需要改为TAB_Aws2022
# tTime是时间，查询对应时间就修改成对应的时段-------》'tTime between '2022-01-10 20:00' and '2022-02-10 08:00''
sql_M_location = 'select RR,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10\
    ,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20,\
    R21,R22,R23,R24,R25,R26,R27,R28,R29,R30,\
    R31,R32,R33,R34,R35,R36,R37,R38,R39,R40,\
    R41,R42,R43,R44,R45,R46,R47,R48,R49,R50,\
    R51,R52,R53,R54,R55,R56,R57,R58,R59,R60 \
    from TAB_Mws2022 as a left join TAB_StationInfo as b on a.IIiii=b.IIiii where\
    (b.IIiii in (select IIiii from TAB_StationInfo where(City = '台州') and\
    tTime between '2022-01-10 20:00' and '2022-02-10 08:00'))'

df_M_location = pd.read_sql(sql_M_location , con=conn)

In [18]:
df_M_location

,RR,R01,R02,R03,R04,R05,R06,R07,R08,R09,...,R51,R52,R53,R54,R55,R56,R57,R58,R59,R60
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338439,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338442,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
